In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
def parse_driver_name(row):
    words = row.split()
    parsed_string = ' '.join(words[:-1])
    return parsed_string

In [3]:
FP1_results = pd.DataFrame()

for year in range(2019, 2021):
    races_url = f'https://www.formula1.com/en/results.html/{year}/races.html'
    response = requests.get(races_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    race_links = []
    filter_links = soup.find_all('a', attrs={'class': 'resultsarchive-filter-item-link FilterTrigger'})
    for link in filter_links:
        href = link.get('href')
        if f'/en/results.html/{year}/races/' in href:
            race_links.append(href)

    year_df = pd.DataFrame()

    round_num = 1
    for race_link in race_links:
        FP1_link = race_link.replace('race-result.html', 'practice-1.html')
        try:
            df = pd.read_html(f'https://www.formula1.com{FP1_link}')[0]
            df = df[['Pos','Driver']]
        except:
            try: 
                ignore = pd.read_html(f'https://www.formula1.com{race_link}')[0]
                round_num+=1
                continue
            except:
                continue
        df['season'] = year
        df['round'] = round_num
        df = df.loc[:, ~df.columns.str.contains('Unnamed')]
        year_df = pd.concat([year_df, df], ignore_index=True)
        round_num += 1

    FP1_results = pd.concat([FP1_results, year_df], ignore_index=True)

print(FP1_results.shape)

(757, 4)


In [4]:
FP1_results["Driver"] = FP1_results["Driver"].apply(parse_driver_name)

In [5]:
print(FP1_results)

     Pos             Driver  season  round
0      1     Lewis Hamilton    2019      1
1      2   Sebastian Vettel    2019      1
2      3    Charles Leclerc    2019      1
3      4     Max Verstappen    2019      1
4      5    Valtteri Bottas    2019      1
..   ...                ...     ...    ...
752   16     George Russell    2020     17
753   17    Nicholas Latifi    2020     17
754   18    Mick Schumacher    2020     17
755   19  Pietro Fittipaldi    2020     17
756   20   Daniel Ricciardo    2020     17

[757 rows x 4 columns]
